# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# PlanLoopController — E2E Notebook (STATIC loop)

## Scope
We validate PlanLoopController behavior end-to-end in STATIC planning mode:
- REPLAN_REQUIRED loop iteration
- replan_ctx injection into EnginePlanner prompt
- bounded replanning (max_replans)
- bounded identical plan repeats (max_same_plan_repeats)
- HITL escalation when limits are exceeded

## Constraints (this session)
- We add **one notebook cell per message** (including its English markdown).
- We use **only Intergrax framework classes** (no fake adapters, no fake classes, no new custom structures).
- A single **Configuration cell** will contain all setup objects (runtime context/state, real LLM adapter, planners, controller, policy).
- Each test will be a **separate code cell**.


## Test 1 — Baseline (single pass, no replan)

Goal:
- Run `PlanLoopController.run_static(...)` once in a normal scenario where execution completes.
- Verify the loop returns a `RuntimeAnswer` with a successful outcome (COMPLETED path).

Notes:
- This test assumes the default StepExecutor registry can complete the plan for a simple user message
  without forcing REPLAN or NEEDS_USER_INPUT.
- If any step is not configured in your environment (e.g., missing adapters/services), the test will
  fail and we will fix it by adjusting the *existing* Intergrax configuration (no fakes).


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint
from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


# Test 1 input (should not require web/tools/rag)

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",    
)


# ---------------------------------------------------------------------
# Runtime context + request + state
# ---------------------------------------------------------------------

session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,    
)

request = RuntimeRequest(
    session_id="planloop-test-session",
    user_id="planloop-test-user",
    message="""
Context:
- Intergrax nexus Runtime uses a planning/execution loop.
- PlanLoopController orchestrates bounded replanning:
  - calls EnginePlanner to get an EnginePlan,
  - StepPlanner builds a plan (STATIC: full plan),
  - StepExecutor executes steps and returns stop_reason,
  - loop repeats on REPLAN_REQUIRED with max_replans and max_same_plan_repeats,
  - escalates to HITL if the loop is stuck.

Task:
Explain what PlanLoopController does in Intergrax, in one short paragraph.
""".strip(),
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run-001",
)
state.configure_llm_tracker()

# Capabilities ON — so the planner is not clamped
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)

# ---------------------------------------------------------------------
# Planning + execution stack
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)

step_planner = StepPlanner(StepPlannerConfig())

registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=3,
    max_same_plan_repeats=2,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

print("Configuration ready.")

answer = await plan_loop.run_static(
    state=state,
    plan_id_prefix="e2e-test-1",
    user_message=state.request.message,
)

print("Answer:", answer.answer)
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)
pprint.pprint([e.to_dict() for e in state.trace_events][-20:])



Configuration ready.
Answer: The PlanLoopController orchestrates bounded replanning in Intergrax nexus Runtime. It iteratively generates an EnginePlan through EnginePlanner and StepPlanner, executes steps with StepExecutor, and upon failures or repetition, it repeats the process within set bounds (max_replans and max_same_plan_repeats) before escalating to Higher-Independent Tasking Logic (HITL) if stuck.
Route.strategy: llm_only
Route.extra: {'used_attachments_context': False, 'attachments_chunks': 0}
{'base_history_length': 1,
 'events': [{'component': 'runtime',
             'data': {},
             'level': 'info',
             'message': 'Step finished',
             'step': 'SessionAndIngestStep',
             'ts_utc': '2026-01-06T16:32:22.553574+00:00'},
            {'component': 'runtime',
             'data': {},
             'level': 'info',
             'message': 'Step started',
             'step': 'ProfileBasedMemoryStep',
             'ts_utc': '2026-01-06T16:32:22.553

## Test 2 — REPLAN_REQUIRED triggers a second loop iteration (E2E, Intergrax-only)

Goal:
- Validate that `PlanLoopController.run_static(...)` performs a replanning iteration when a step returns `REPLAN_REQUIRED`.

What we verify:
1) The execution triggers `REPLAN_REQUIRED` (via a real StepExecutor registry override for `VERIFY_ANSWER`).
2) The loop performs at least one replanning iteration (i.e., it runs more than one planning/execution cycle).
3) The final outcome is not a silent failure (either it completes after replanning, or it escalates based on policy).

Constraints:
- No fake adapters / no fake classes.
- We only use Intergrax components.
- This cell is autonomous: it creates its own config, context, state, registry, and controller.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.stepplan_models import ExecutionStep, StepAction
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepHandlerRegistry,
    StepReplanRequested,
    ReplanCode,
    ReplanReason,
)


# ---------------------------------------------------------------------
# Config (RAG/web/tools off for determinism)
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-test-session-t2",
    user_id="planloop-test-user",
    message="""
Context:
- Intergrax uses PlanLoopController: EnginePlanner -> StepPlanner(STATIC) -> StepExecutor.
Task:
Return a short summary of PlanLoopController behavior.
""".strip(),
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run-t2",
)
state.configure_llm_tracker()

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)


# ---------------------------------------------------------------------
# Build planning stack
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()

# Override handler: VERIFY_ANSWER -> force replan via Intergrax-native exception
async def verify_force_replan(step: ExecutionStep, exec_ctx: RuntimeState):
    raise StepReplanRequested(
        code=ReplanCode.STEP_POLICY_REPLAN,
        reason=ReplanReason.UNSPECIFIED,
        details={
            "test": "planloop_e2e_force_replan",
            "step_id": step.step_id.value,
            "action": step.action.value,
        },
    )

handlers = dict(base_registry.handlers)
handlers[StepAction.VERIFY_ANSWER] = verify_force_replan
registry = StepHandlerRegistry(handlers=handlers)

step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=1,            # force a second iteration, then exceed
    max_same_plan_repeats=99, # not testing fingerprint here
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

print("Test 2 configuration ready.")


# ---------------------------------------------------------------------
# Run (we expect a RuntimeError after the second iteration)
# ---------------------------------------------------------------------

try:
    answer = await plan_loop.run_static(
        state=state,
        plan_id_prefix="e2e-test-2",
        user_message=state.request.message,
    )
    print("UNEXPECTED: controller returned answer (expected RuntimeError due to max_replans).")
    print("Answer:", answer.answer)
    print("Route.strategy:", answer.route.strategy)
    print("Route.extra:", answer.route.extra)

except RuntimeError as e:
    print("EXPECTED RuntimeError:", str(e))

print("\n--- Debug trace (tail) ---")
pprint.pprint([e.to_dict() for e in state.trace_events][-20:])


Test 2 configuration ready.
EXPECTED RuntimeError: PlanLoopController(STATIC): max replans exceeded. max_replans=1 replans_used=2 replan_reason='step_policy_replan: unspecified'

--- Debug trace (tail) ---
[{'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step started',
  'step': 'ProfileBasedMemoryStep',
  'ts_utc': '2026-01-06T16:32:34.081032+00:00'},
 {'component': 'engine',
  'data': {'enable_org_profile_memory': True,
           'enable_user_profile_memory': True,
           'has_org_profile_instructions': False,
           'has_user_profile_instructions': False},
  'level': 'info',
  'message': 'Profile-based instructions loaded for session.',
  'step': 'memory_layer',
  'ts_utc': '2026-01-06T16:32:34.081032+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step': 'ProfileBasedMemoryStep',
  'ts_utc': '2026-01-06T16:32:34.081032+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message'

## Test 3 — Strong assertions (max_replans + replan_reason)

Goal:
- Convert Test 2 into a deterministic, self-checking test (not just printing output).
- Assert that PlanLoopController:
  1) exceeded `max_replans`,
  2) reports correct `max_replans` and `replans_used` values,
  3) includes the replan reason that originates from the StepExecutor replan request.

This proves (a) the loop actually performed replanning iterations, and (b) the stop condition is enforced by PlanLoopPolicy, not by incidental runtime failures.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.stepplan_models import StepAction, ExecutionStep
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepHandlerRegistry,
    StepReplanRequested,
    ReplanCode,
    ReplanReason,
)


# ---------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-test-session-t2-assert",
    user_id="planloop-test-user",
    message="""
Context:
- Intergrax uses PlanLoopController: EnginePlanner -> StepPlanner(STATIC) -> StepExecutor.
Task:
Return a short summary of PlanLoopController behavior.
""".strip(),
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run-t2-assert",
)
state.configure_llm_tracker()

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)


# ---------------------------------------------------------------------
# Planning stack
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()

async def verify_force_replan(step: ExecutionStep, exec_ctx: RuntimeState) -> None:
    raise StepReplanRequested(
        code=ReplanCode.STEP_POLICY_REPLAN,
        reason=ReplanReason.UNSPECIFIED,
        details={
            "test": "planloop_e2e_force_replan",
            "step_id": step.step_id.value,
            "action": step.action.value,
        },
    )

handlers = dict(base_registry.handlers)
handlers[StepAction.VERIFY_ANSWER] = verify_force_replan
registry = StepHandlerRegistry(handlers=handlers)

step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=1,
    max_same_plan_repeats=99,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

print("Test 3 (assert) configuration ready.")


# ---------------------------------------------------------------------
# Run + assertions
# ---------------------------------------------------------------------

error_text = None
try:
    _ = await plan_loop.run_static(
        state=state,
        plan_id_prefix="e2e-test-2-assert",
        user_message=state.request.message,
    )
    raise AssertionError("Expected RuntimeError due to max_replans exceeded, but run_static returned normally.")

except RuntimeError as e:
    error_text = str(e)

print("RuntimeError:", error_text)

# Strong assertions on the error message (stable contract)
assert "max replans exceeded" in error_text
assert "max_replans=1" in error_text
assert "replans_used=2" in error_text
assert "step_policy_replan" in error_text

print("OK: max_replans contract validated.")

# Optional: show tail debug trace (diagnostics only)
print("\n--- Debug trace (tail) ---")
pprint.pprint([e.to_dict() for e in state.trace_events][-10:])


Test 3 (assert) configuration ready.
RuntimeError: PlanLoopController(STATIC): max replans exceeded. max_replans=1 replans_used=2 replan_reason='step_policy_replan: unspecified'
OK: max_replans contract validated.

--- Debug trace (tail) ---
[{'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step started',
  'step': 'EnsureCurrentUserMessageStep',
  'ts_utc': '2026-01-06T16:32:45.781749+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step': 'EnsureCurrentUserMessageStep',
  'ts_utc': '2026-01-06T16:32:45.781749+00:00'},
 {'component': 'planner',
  'data': {'has_replan_ctx': False,
           'replan_reason': None,
           'replans_used': 0,
           'same_plan_repeats': 0},
  'level': 'info',
  'message': 'Planning iteration started.',
  'step': 'plan_loop_static',
  'ts_utc': '2026-01-06T16:32:45.783322+00:00'},
 {'component': 'planner',
  'data': {'ask_clarifying_question': False,
           'capability_c

## Test 4 — NEEDS_USER_INPUT (HITL / clarify path)

## Discover HITL/User Input control-flow primitives in Intergrax

Goal:
- Identify the Intergrax-native exception/result used to signal NEEDS_USER_INPUT from a step handler.
- We will inspect `step_executor_models` for available classes/enums related to user input / HITL / clarify.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.stepplan_models import StepAction, ExecutionStep
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepExecutionResult,
    StepStatus,
    StepHandlerRegistry,
)

# ---------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(config=config, session_manager=session_manager)

request = RuntimeRequest(
    session_id="planloop-test-session-t3",
    user_id="planloop-test-user",
    message="Force clarify test",
    attachments=[],
)

state = RuntimeState(context=ctx, request=request, run_id="planloop-e2e-run-t3")
state.configure_llm_tracker()

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)

# ---------------------------------------------------------------------
# Planning stack
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()

async def clarify_ok(step: ExecutionStep, exec_ctx) -> StepExecutionResult:
    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        sequence=0,
        started_at_utc="",
        ended_at_utc="",
        duration_ms=0,
    )

handlers = dict(base_registry.handlers)
handlers[StepAction.ASK_CLARIFYING_QUESTION] = clarify_ok
registry = StepHandlerRegistry(handlers=handlers)

step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(max_replans=5, max_same_plan_repeats=5)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

print("Test 4 configuration ready.")

# ---------------------------------------------------------------------
# Run
# ---------------------------------------------------------------------

answer = await plan_loop.run_static(
    state=state,
    plan_id_prefix="e2e-test-3",
    user_message="Please ask me a clarifying question.",
)

print("Answer:", answer.answer)
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)

print("\n--- Debug trace (tail) ---")
pprint.pprint([e.to_dict() for e in state.trace_events][-10:])


Test 4 configuration ready.
Answer: Could you clarify what you mean and what outcome you want?
Route.strategy: hitl_clarify
Route.extra: {'stop_reason': 'needs_user_input', 'origin_step_id': <StepId.CLARIFY: 'clarify'>, 'context_key': 'clarify.user_input', 'must_answer_to_continue': True}

--- Debug trace (tail) ---
[{'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step': 'BuildBaseHistoryStep',
  'ts_utc': '2026-01-06T16:32:57.230946+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step started',
  'step': 'HistoryStep',
  'ts_utc': '2026-01-06T16:32:57.230946+00:00'},
 {'component': 'engine',
  'data': {'base_history_length': 1,
           'history_includes_current_user': True,
           'history_length': 1},
  'level': 'info',
  'message': 'Conversation history built for LLM.',
  'step': 'history',
  'ts_utc': '2026-01-06T16:32:57.230946+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  '

## Test 5 — Same plan repeats → HITL escalation (deterministic, forced_plan)

Goal:
- Make the planning loop deterministic (no LLM flakiness).
- Force StepExecutor to request REPLAN_REQUIRED on every iteration.
- Use EnginePlanner forced_plan to produce the same EnginePlan fingerprint each time.
- Expect HITL escalation due to same plan repeating beyond the limit.

Assertions:
- answer.route.strategy == "hitl_escalation_same_plan_repeat"
- answer.route.extra["stop_reason"] == "NEEDS_USER_INPUT"
- answer.route.extra["same_plan_repeats"] >= answer.route.extra["same_plan_repeat_limit"]
- answer.route.extra["same_plan_repeat_limit"] == policy.max_same_plan_repeats

In [ ]:
import sys, os

from intergrax.runtime.nexus.tracing.plan.planner_build_debug import PlannerBuildDebugDiagV1
from intergrax.runtime.nexus.tracing.trace_models import TraceComponent
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.stepplan_models import ExecutionStep, StepAction
from intergrax.runtime.nexus.planning.engine_plan_models import EnginePlan, PlannerPromptConfig
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepHandlerRegistry,
    StepReplanRequested,
    ReplanCode,
    ReplanReason,
)

# ---------------------------------------------------------------------
# Deterministic forced EnginePlan (same fingerprint every iteration)
# ---------------------------------------------------------------------
planner_prompt_config = PlannerPromptConfig(
    forced_plan=EnginePlan.generic_synthesize(
        version="forced-v1",
        reasoning_summary="forced plan for deterministic same-plan-repeat test",
        use_user_longterm_memory=False,
    )
)

# ---------------------------------------------------------------------
# Config (RAG/web/tools off for determinism)
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    planner_prompt_config=planner_prompt_config,  # IMPORTANT
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-test-session-t4",
    user_id="planloop-test-user",
    message="Test 4: Force same plan repeats and expect HITL escalation.",
)

state = RuntimeState(
    request=request,
    context=ctx,
    run_id="planloop-e2e-run-t4",
)
state.configure_llm_tracker()

# Capabilities (not critical here, but keep consistent)
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)

# ---------------------------------------------------------------------
# Build planning stack
# ---------------------------------------------------------------------
engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()
handlers = dict(base_registry.handlers)

# Force REPLAN_REQUIRED deterministically in one real handler
async def verify_force_replan(step: ExecutionStep, exec_ctx):
    raise StepReplanRequested(
        code=ReplanCode.STEP_POLICY_REPLAN,
        reason=ReplanReason.UNSPECIFIED,
        details={
            "message": "Forced REPLAN_REQUIRED for same-plan-repeat test.",
            "step_id": step.step_id.value,
            "action": step.action.value,
        },
    )

handlers[StepAction.VERIFY_ANSWER] = verify_force_replan

registry = StepHandlerRegistry(handlers=handlers)
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=99,            # make sure max_replans does NOT mask this scenario
    max_same_plan_repeats=1,   # escalate on the first repeated fingerprint
    # on_max_replans="raise" or "hitl" (does not matter here)
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

# ---------------------------------------------------------------------
# Run (expect HITL escalation due to same plan repeats)
# ---------------------------------------------------------------------
answer = await plan_loop.run_static(
    state=state,
    plan_id_prefix="e2e-test-4",
    user_message=state.request.message,
)

print("Answer:", answer.answer)
print("Route.strategy:", answer.route.strategy)
print("Route.extra:")
pprint.pprint(answer.route.extra)

assert answer.route.strategy == "hitl_escalation_same_plan_repeat"
assert answer.route.extra.get("stop_reason") == "NEEDS_USER_INPUT"

repeat_limit = answer.route.extra.get("same_plan_repeat_limit")
same_repeats = answer.route.extra.get("same_plan_repeats")

assert repeat_limit == policy.max_same_plan_repeats
assert same_repeats is not None and same_repeats >= repeat_limit

forced_used = False
forced_hash = None

for ev in state.trace_events:
    if (
        ev.component == TraceComponent.PLANNER
        and ev.step == "engine_planner"
        and ev.message == "Planner build debug."
        and isinstance(ev.payload, PlannerBuildDebugDiagV1)
    ):
        if ev.payload.planner_forced_plan_used is True:
            forced_used = True
            forced_hash = ev.payload.planner_forced_plan_hash
            break

assert forced_used, "Expected planner_forced_plan_used=True in Planner build debug event."
print("forced_plan hash:", forced_hash)

Answer: I need one clarification to continue. Replan reason: step_policy_replan: unspecified Please clarify what you want me to do next.
Route.strategy: hitl_escalation_same_plan_repeat
Route.extra:
{'last_fingerprint': 'f7189388b4dab870758a284d8f3bed00d6ff49038fe77644f07fcefab05f93d0',
 'replans_used': 1,
 'same_plan_repeat_limit': 1,
 'same_plan_repeats': 1,
 'stop_reason': 'NEEDS_USER_INPUT'}
forced_plan hash: b97da21fc94a5028


## Test 5 — FAILED path (deterministic)

Goal:
- Force a hard RuntimeError inside a real step handler.
- Expect PlanLoopController(STATIC) to terminate deterministically with an exception (FAILED path).

Assertions:
- RuntimeError is raised by PlanLoopController.run_static().
- Error message contains "plan execution FAILED" (or equivalent invariant).


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.stepplan_models import ExecutionStep, StepAction
from intergrax.runtime.nexus.planning.engine_plan_models import PlannerPromptConfig, EnginePlan
from intergrax.runtime.nexus.planning.step_executor_models import StepHandlerRegistry

# ---------------------------------------------------------------------
# Deterministic forced EnginePlan (avoid LLM flakiness)
# ---------------------------------------------------------------------
planner_prompt_config = PlannerPromptConfig(
    forced_plan=EnginePlan.generic_synthesize(
        reasoning_summary="forced plan for deterministic FAILED-path test"
    )
)

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    planner_prompt_config=planner_prompt_config,
)

session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)

request = RuntimeRequest(
    session_id="planloop-test-session-t5",
    user_id="planloop-test-user",
    message="Test 5: Force FAILED path (handler RuntimeError).",
    attachments=[],
)

state = RuntimeState(
    request=request,
    context=ctx,
    run_id="planloop-e2e-run-t5",
)
state.configure_llm_tracker()

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)

# ---------------------------------------------------------------------
# Build planning stack
# ---------------------------------------------------------------------
engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()
handlers = dict(base_registry.handlers)

# Force a hard failure in one real handler.
# FINALIZE_ANSWER is ideal because the loop expects runtime_answer to be set there.
async def finalize_force_fail(step: ExecutionStep, exec_ctx):
    raise RuntimeError("Forced failure in FINALIZE_ANSWER (Test 5).")

handlers[StepAction.FINALIZE_ANSWER] = finalize_force_fail

registry = StepHandlerRegistry(handlers=handlers)
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=3,
    max_same_plan_repeats=99,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

# ---------------------------------------------------------------------
# Run (expect RuntimeError)
# ---------------------------------------------------------------------
try:
    _ = await plan_loop.run_static(
        state=state,
        plan_id_prefix="e2e-test-5",
        user_message=state.request.message,
    )
    raise AssertionError("UNEXPECTED: controller returned answer (expected RuntimeError).")
except RuntimeError as e:
    msg = str(e)
    print("OK: RuntimeError raised as expected.")
    print("Error:", msg)
    assert "FAILED" in msg or "plan execution" in msg

print("\n--- Debug trace (tail) ---")
pprint.pprint([e.to_dict() for e in state.trace_events][-20:])



OK: RuntimeError raised as expected.
Error: PlanLoopController(STATIC): plan execution FAILED. step_id=final action=FINALIZE_ANSWER error=handler_exception: Forced failure in FINALIZE_ANSWER (Test 5).

--- Debug trace (tail) ---
[{'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step': 'SessionAndIngestStep',
  'ts_utc': '2026-01-06T16:33:01.011465+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step started',
  'step': 'ProfileBasedMemoryStep',
  'ts_utc': '2026-01-06T16:33:01.011465+00:00'},
 {'component': 'engine',
  'data': {'enable_org_profile_memory': True,
           'enable_user_profile_memory': True,
           'has_org_profile_instructions': False,
           'has_user_profile_instructions': False},
  'level': 'info',
  'message': 'Profile-based instructions loaded for session.',
  'step': 'memory_layer',
  'ts_utc': '2026-01-06T16:33:01.011465+00:00'},
 {'component': 'runtime',
  'data': {},
  'level'

## Test 0 — Happy Path (COMPLETED) with forced_plan

Goal:
- Use deterministic EnginePlanner via forced_plan.
- Execute full default STATIC plan end-to-end with no handler overrides.
- Expect controller to return RuntimeAnswer (COMPLETED path), not HITL and not exceptions.

Assertions:
- answer is RuntimeAnswer and answer.answer is non-empty string
- route.strategy is not hitl_clarify / not hitl_escalation_*
- forced_plan usage is visible in trace (engine_planner event)


In [ ]:
import sys, os

from intergrax.runtime.nexus.tracing.plan.planner_build_debug import PlannerBuildDebugDiagV1
from intergrax.runtime.nexus.tracing.trace_models import TraceComponent
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.pipelines.contract import RuntimePipeline
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner
from intergrax.runtime.nexus.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest
from intergrax.runtime.nexus.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.nexus.runtime_steps.setup_steps_tool import SETUP_STEPS
from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager

from intergrax.runtime.nexus.planning.engine_plan_models import PlannerPromptConfig, EnginePlan

# ---------------------------------------------------------------------
# Deterministic forced EnginePlan (avoid planner LLM flakiness)
# ---------------------------------------------------------------------
planner_prompt_config = PlannerPromptConfig(
    forced_plan=EnginePlan.generic_synthesize(
        reasoning_summary="forced plan for deterministic HAPPY PATH test"
    )
)


# ---------------------------------------------------------------------
# Config (RAG/web/tools off for determinism; tools_mode off)
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    planner_prompt_config=planner_prompt_config,
)

session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)

request = RuntimeRequest(
    session_id="planloop-test-session-happy",
    user_id="planloop-test-user",
    message=(
        "Happy path test: Please explain (briefly) what PlanLoopController does in STATIC mode "
        "in Intergrax nexus Runtime."
    ),
    attachments=[],
)

state = RuntimeState(
    request=request,
    context=ctx,
    run_id="planloop-e2e-run-happy",
)
state.configure_llm_tracker()

# Capabilities (keep consistent)
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)

# ---------------------------------------------------------------------
# Build planning stack (default handlers)
# ---------------------------------------------------------------------
engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)
step_planner = StepPlanner(StepPlannerConfig())

base_registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=base_registry)

policy = PlanLoopPolicy(
    max_replans=2,
    max_same_plan_repeats=3,
    # on_max_replans default is fine here
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

# ---------------------------------------------------------------------
# Run (expect COMPLETED -> RuntimeAnswer returned)
# ---------------------------------------------------------------------
answer = await plan_loop.run_static(
    state=state,
    plan_id_prefix="e2e-happy",
    user_message=state.request.message,
)

print("Answer length:", len(answer.answer or ""))
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)

# Core assertions (do NOT depend on exact LLM content)
assert isinstance(answer.answer, str) and len(answer.answer.strip()) > 0

# Should not be any HITL routes on happy path
assert answer.route.strategy not in (
    "hitl_clarify",
    "hitl_escalation_same_plan_repeat",
    "hitl_escalation_max_replans",
)

# forced_plan should be observable in trace (EnginePlanner event)
forced_used = False
forced_hash = None

for ev in state.trace_events:
    if (
        ev.component == TraceComponent.PLANNER
        and ev.step == "engine_planner"
        and ev.message == "Planner build debug."
        and isinstance(ev.payload, PlannerBuildDebugDiagV1)
    ):
        if ev.payload.planner_forced_plan_used is True:
            forced_used = True
            forced_hash = ev.payload.planner_forced_plan_hash
            break

assert forced_used, "Expected forced_plan usage in engine_planner trace event."
print("forced_plan hash:", forced_hash)

print("\n--- Debug trace (tail) ---")
pprint.pprint([e.to_dict() for e in state.trace_events][-25:])


Answer length: 946
Route.strategy: llm_only
Route.extra: {'used_attachments_context': False, 'attachments_chunks': 0}
forced_plan hash: 7b925484c8c87447

--- Debug trace (tail) ---
[{'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step': 'SessionAndIngestStep',
  'ts_utc': '2026-01-06T16:37:41.357992+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step started',
  'step': 'ProfileBasedMemoryStep',
  'ts_utc': '2026-01-06T16:37:41.357992+00:00'},
 {'component': 'engine',
  'data': {'enable_org_profile_memory': True,
           'enable_user_profile_memory': True,
           'has_org_profile_instructions': False,
           'has_user_profile_instructions': False},
  'level': 'info',
  'message': 'Profile-based instructions loaded for session.',
  'step': 'memory_layer',
  'ts_utc': '2026-01-06T16:37:41.357992+00:00'},
 {'component': 'runtime',
  'data': {},
  'level': 'info',
  'message': 'Step finished',
  'step'